In [1]:
import torch
import torchaudio
from torchaudio.pipelines import MMS_FA as bundle
from typing import List
import IPython
import matplotlib.pyplot as plt
from pypinyin import pinyin, lazy_pinyin, Style

print(torch.__version__)
print(torchaudio.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import whisper
from pydub import AudioSegment

2.1.2+cu118
2.1.2+cu118
cuda


In [2]:
raw_data = "raw_audio\\test4.wav"

model = whisper.load_model("large-v2")
# 使用Whisper進行語音識別
result = model.transcribe(raw_data)
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 3.58,
  'text': '最後再把這兩個圖形同時框圈起來',
  'tokens': [50364,
   20578,
   8623,
   16075,
   2664,
   34623,
   2523,
   244,
   30900,
   13089,
   6611,
   21416,
   228,
   2523,
   230,
   21670,
   50543],
  'temperature': 0.0,
  'avg_logprob': -0.17178617396824797,
  'compression_ratio': 1.3898916967509025,
  'no_speech_prob': 0.01168491318821907},
 {'id': 1,
  'seek': 0,
  'start': 3.58,
  'end': 7.94,
  'text': '到左上角點擊合併圖案 執行剪去',
  'tokens': [50543,
   4511,
   21975,
   5708,
   29389,
   8216,
   37813,
   14245,
   1593,
   113,
   2523,
   244,
   28899,
   220,
   16262,
   115,
   8082,
   5935,
   103,
   6734,
   50761],
  'temperature': 0.0,
  'avg_logprob': -0.17178617396824797,
  'compression_ratio': 1.3898916967509025,
  'no_speech_prob': 0.01168491318821907},
 {'id': 2,
  'seek': 0,
  'start': 7.94,
  'end': 9.98,
  'text': '這樣就會在原本的矩形上',
  'tokens': [50761,
   8377,
   47778,
   3581,
   19683,
   8802,
   1546,
   5881,


In [3]:
sentence =[]
for i in range(len(result['segments'])):
    sentence.append([result['segments'][i]['text'].lower().replace(',', ''), result['segments'][i]['start'], result['segments'][i]['end']])
    

sentence

[['最後再把這兩個圖形同時框圈起來', 0.0, 3.58],
 ['到左上角點擊合併圖案 執行剪去', 3.58, 7.94],
 ['這樣就會在原本的矩形上', 7.94, 9.98],
 ['這樣第一個字的預備工作就算是完成了', 9.98, 13.06],
 ['至於第二個字分的做法也是一樣的', 13.06, 16.12],
 ['同樣是在分字的後方建立一個矩形', 16.12, 18.94],
 ['然後把兩個圖形進行相剪產生一塊模板', 18.94, 22.28],
 ['最後再把模板利用剪下貼上的快捷鍵移到第一張投影片', 22.28, 27.14],
 ['最後我們稍微調整一下兩塊模板的位置和大小', 27.46, 31.02],
 ['來將這兩個字上下並排', 31.02, 33.34],
 ['放置在待會文字動畫出現的地方', 33.34, 36.02],
 ['營造出一種懷舊復古的風情', 36.02, 38.66],
 ['裡面收藏了許多印章的風格', 38.66, 40.980000000000004],
 ['點擊印章內的文字', 40.980000000000004, 42.980000000000004],
 ['再按一下底部的轉換按鈕', 42.980000000000004, 45.36],
 ['好 我重複這個流程繼續修改印章的第二個字', 45.36, 48.86],
 ['字型同樣是套用明朝體', 48.86, 51.14],
 ['顏色設為紅色', 51.14, 52.980000000000004],
 ['這樣我們就能把這個印章下載到自己的電腦', 52.980000000000004, 56.18],
 ['放置到標題文字的右下角', 56.26, 58.26],
 ['ok 那我們可以準備來製作手寫文字的動畫囉', 59.06, 62.06]]

In [4]:
audio = AudioSegment.from_file("raw_audio\\test4.wav")
for i in range(len(sentence)):
    audio_clip = audio[sentence[i][1] *1000: sentence[i][2]*1000]
    audio_clip.export(f"sentences\\{(sentence[i][0].lower())}.wav", format="wav")

In [5]:
model = bundle.get_model()
model.to(device)

tokenizer = bundle.get_tokenizer()
aligner = bundle.get_aligner()

def compute_alignments(waveform: torch.Tensor, transcript: List[str]):
    with torch.inference_mode():
        emission, _ = model(waveform.to(device))
        token_spans = aligner(emission[0], tokenizer(transcript))
    return emission, token_spans

def _score(spans):
    return sum(s.score * len(s) for s in spans) / sum(len(s) for s in spans)


def plot_alignments(waveform, token_spans, emission, transcript, sample_rate=bundle.sample_rate):
    ratio = waveform.size(1) / emission.size(1) / sample_rate

    fig, axes = plt.subplots(2, 1)
    axes[0].imshow(emission[0].detach().cpu().T, aspect="auto")
    axes[0].set_title("Emission")
    axes[0].set_xticks([])

    axes[1].specgram(waveform[0], Fs=sample_rate)
    for t_spans, chars in zip(token_spans, transcript):
        t0, t1 = t_spans[0].start, t_spans[-1].end
        axes[0].axvspan(t0 - 0.5, t1 - 0.5, facecolor="None", hatch="/", edgecolor="white")
        axes[1].axvspan(ratio * t0, ratio * t1, facecolor="None", hatch="/", edgecolor="white")
        axes[1].annotate(f"{_score(t_spans):.2f}", (ratio * t0, sample_rate * 0.51), annotation_clip=False)

        for span, char in zip(t_spans, chars):
            t0 = span.start * ratio
            axes[1].annotate(char, (t0, sample_rate * 0.55), annotation_clip=False)

    axes[1].set_xlabel("time [second]")
    fig.tight_layout()

def preview_word(waveform, spans, num_frames, transcript, sample_rate):
    ratio = waveform.size(1) / num_frames
    x0 = int(ratio * spans[0].start)
    x1 = int(ratio * spans[-1].end)
    #print(f"{transcript} ({_score(spans):.2f}): {x0 / sample_rate:.3f} - {x1 / sample_rate:.3f} sec")
    time_StarAndEnd = [ x0 / sample_rate, x1/sample_rate] # 回傳單個字的起始時間與結束時間
    segment = waveform[:, x0:x1]
    #return IPython.display.Audio(segment.numpy(), rate=sample_rate)
    return time_StarAndEnd
    

In [10]:
for i in range(len(sentence)):#len(sentence)-1
    text_normalized = ' '.join(lazy_pinyin(sentence[i][0]))#將文字轉為沒有音調的拼音，lazy_pinyin是陣列所以要再join成字串

    waveform, sample_rate = torchaudio.load(f"sentences\\{sentence[i][0]}.wav")
    waveform = waveform[0:1]

    transcript = text_normalized.split()
    emission, token_spans = compute_alignments(waveform, transcript)
    num_frames = emission.size(1)


    #plot_alignments(waveform, token_spans, emission, transcript)

    print("Raw Transcript: ", sentence[i][0])
    print("Normalized Transcript: ", text_normalized)
    IPython.display.Audio(waveform, rate=sample_rate)

    text_raw = sentence[i][0]
    word_start_end = []
    pinyin_tone = pinyin(text_raw, style=Style.TONE3, heteronym=False)
    for j in range(len(transcript)):#len(transcript)
        timeStartEnd = preview_word(waveform, token_spans[j], num_frames, transcript[j], sample_rate)
        word_start_end.append([pinyin_tone[j][0], timeStartEnd[0], timeStartEnd[1]])
    print(word_start_end)

    audio = AudioSegment.from_file(f"sentences\\{sentence[i][0]}.wav")
    file_name = sentence[i][0]
    for k in range(len(word_start_end)):
        segment_audio = audio[word_start_end[k][1] *1000: word_start_end[k][2]*1000]
        segment_audio.export(f"data\\{file_name}-{k}_{word_start_end[k][0]}.wav", format="wav")
    print('------------------------------------------')

Raw Transcript:  最後再把這兩個圖形同時框圈起來
Normalized Transcript:  zui hou zai ba zhe liang ge tu xing tong shi kuang quan qi lai
[['zui4', 0.6970975056689342, 0.806031746031746], ['hou4', 0.8350793650793651, 1.0238775510204081], ['zai4', 1.0674603174603174, 1.1473242630385487], ['ba3', 1.2126757369614511, 1.2562585034013605], ['zhe4', 1.4668480725623583, 1.517687074829932], ['liang3', 1.56124716553288, 1.6992290249433106], ['ge4', 1.706485260770975, 1.7282539682539682], ['tu2', 1.779092970521542, 1.9170748299319729], ['xing2', 1.967891156462585, 2.1276643990929704], ['tong2', 2.178480725623583, 2.2874149659863945], ['shi2', 2.3600226757369613, 2.4181179138321998], ['kuang1', 2.548843537414966, 2.679546485260771], ['quan1', 2.7013378684807257, 2.8029931972789117], ['qi3', 2.8320408163265305, 2.868344671201814], ['lai2', 2.897392290249433, 3.0426303854875285]]
------------------------------------------
Raw Transcript:  到左上角點擊合併圖案 執行剪去
Normalized Transcript:  dao zuo shang jiao dian ji he bing tu 